<a href="https://colab.research.google.com/github/liz-lewis-manchester/CNM_2025_group_01/blob/Compute-solution-to-array/Compute_solution_to_array.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd

#1. Helper Function: Pure In-Memory Interpolation

def interpolate_initial_conditions(L, dx, x_data, theta_data):
    """
    Interpolates discrete data points onto the solver grid.
    No file reading involved - takes arrays directly.
    """
    # Create the solver's spatial grid
    x_grid = np.arange(0, L + dx, dx)

    # NumPy Linear interpolation
    # np.interp(x_new, x_known, y_known)
    theta_initial = np.interp(x_grid, x_data, theta_data)

    # Physics constraint: concentration >= 0
    theta_initial[theta_initial < 0] = 0

    return x_grid, theta_initial


#2. Core Solver Class (Standard BTBS)

class AdvectionSolver:
    """
    Finite difference solver for the 1D advection equation: d(theta)/dt + U*d(theta)/dx = 0
    This version always uses the Implicit Upwind Scheme (Standard BTBS).
    """

    def __init__(self, L, dx, dt, T, U):
        self.L = L
        self.dx = dx
        self.dt = dt
        self.T = T
        self.U = U

        # Spatial grid
        self.x_grid = np.arange(0, self.L + self.dx, self.dx)
        self.num_points = len(self.x_grid)
        self.num_time_steps = int(self.T / self.dt)

        if self.U <= 0:
            raise ValueError("This solver assumes flow velocity U > 0.")

    def _solve_implicit(self, theta_initial, boundary_theta):
        """
        Implicit Upwind Scheme (Standard BTBS).
        Unconditionally stable.
        Equation: theta_i^{n+1} = (theta_i^n + C * theta_{i-1}^{n+1}) / (1 + C)
        """
        print(f"Using **Implicit Scheme (Standard BTBS)**.")

        theta = theta_initial.copy()
        N = self.num_points

        theta_history = np.zeros((self.num_time_steps + 1, N))
        theta_history[0, :] = theta

        C = abs(self.U) * self.dt / self.dx # Parameter C for the implicit formula
        divisor = 1.0 + C

        for n in range(self.num_time_steps):
            theta_next = np.zeros_like(theta)

            # Boundary Condition
            theta_next[0] = boundary_theta

            # Forward Substitution
            for i in range(1, N):
                numerator = theta[i] + C * theta_next[i-1]
                theta_next[i] = numerator / divisor

            theta = theta_next
            theta_history[n+1, :] = theta

        return theta_history

    def compute_solution(self, theta_initial, boundary_theta):
        # Always use the implicit scheme as requested
        return self._solve_implicit(theta_initial, boundary_theta)


# --- 3. Execution Block ---

if __name__ == '__main__':

    # Test Case A: Implicit scheme for a step function
    print("## Test Case A: Implicit Scheme for Step Function ##")

    L = 20.0; dx = 0.2; dt = 1.0; T = 50.0; U = 0.1

    # Define initial conditions manually
    x_grid_A = np.arange(0, L + dx, dx)
    theta_initial_A = np.zeros_like(x_grid_A)
    theta_initial_A[x_grid_A < 5.0] = 100.0 # Step function

    solver_A = AdvectionSolver(L, dx, dt, T, U)
    res_A = solver_A.compute_solution(theta_initial_A, boundary_theta=100.0)

    print(f"Final Max Concentration: {res_A[-1].max():.2f}")

    # Test Case B: Implicit scheme for custom data (High CFL setup)
    print("## Test Case B: Implicit Scheme for Custom Data ##")

    # High CFL setup
    # dx = 0.05, dt = 10.0, U = 0.1
    L = 20.0; dx = 0.05; dt = 10.0; T = 300.0; U = 0.1

    try:
        # 1. Import data from CSV files using Pandas
        df_data = pd.read_csv('initial_conditions (1).csv', encoding='latin1')

        # 2. Extract data and convert it to a NumPy array
        known_x_data = df_data['Distance (m)'].to_numpy()
        known_theta_data = df_data['Concentration (µg/m_ )'].to_numpy()

        print(" Data successfully imported from file/database.")

    except FileNotFoundError:
        # Extract data and convert it to a NumPy array
        print(" CSV file not found. Using hardcoded test data for Test Case B.")
        known_x_data = np.array([0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0])
        known_theta_data = np.array([300.0,10.0,10.0,10.0,10.0,8.0,8.0,8.0,8.0,7.0,7.0,7.0,35.0,57.0,80.0,85.0,80.0,50.0,40.0,20.0,10.0])
    # Interpolate using the helper function
    x_grid_B, theta_initial_B = interpolate_initial_conditions(
        L, dx, known_x_data, known_theta_data
    )

    solver_B = AdvectionSolver(L, dx, dt, T, U)
    res_B = solver_B.compute_solution(theta_initial_B, boundary_theta=theta_initial_B[0])

    print("\nResult Sample (First 10 points at final time):")
    print(f"X : {solver_B.x_grid[:10].round(2)}")
    print(f"C : {res_B[-1, :10].round(2)}")

## Test Case A: Implicit Scheme for Step Function ##
Using **Implicit Scheme (Standard BTBS)**.
Final Max Concentration: 100.00
## Test Case B: Implicit Scheme for Custom Data ##
 Data successfully imported from file/database.
Using **Implicit Scheme (Standard BTBS)**.

Result Sample (First 10 points at final time):
X : [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45]
C : [300. 300. 300. 300. 300. 300. 300. 300. 300. 300.]
